In [1]:
import numpy as np
from shapely.geometry import Polygon, Point
from sklearn.cluster import KMeans
from scipy.optimize import linear_sum_assignment
from joblib import Parallel, delayed
import math

def asignar_repas_envios(vertices_poligono, cantidad_repas, cantidad_envios, n_clusters, random_state=None, n_jobs=-1):
    """
    Asigna repartidores a envíos utilizando clustering y minimización de distancias geográficas.
    Utiliza paralelización para mejorar los tiempos de cálculo de la matriz de distancias.
    
    Parámetros:
    - vertices_poligono: Lista de tuplas (longitud, latitud) que representan los vértices del polígono.
    - cantidad_repas: Número de repartidores a generar.
    - cantidad_envios: Número de envíos a generar.
    - n_clusters: Número inicial de clusters para KMeans.
    - random_state: Semilla para la generación de números aleatorios (int, None o np.random.RandomState).
    - n_jobs: Número de trabajos en paralelo para joblib (-1 utiliza todos los núcleos disponibles).
    
    Retorna:
    - asignaciones: Lista de tuplas (repa_id, envio_id).
    - sobrantes_repas: Lista de IDs de repartidores no asignados.
    - sobrantes_envios: Lista de IDs de envíos no asignados.
    - total_asignaciones: Número total de asignaciones realizadas.
    - distancia_total_metros: Suma total de las distancias de todas las asignaciones en metros.
    """
    # Crear el polígono
    poligono = Polygon(vertices_poligono)

    # Crear un generador de números aleatorios
    rng = np.random.default_rng(random_state)

    # Función para generar puntos aleatorios dentro del polígono
    def generar_puntos_en_poligono(poligono, cantidad):
        minx, miny, maxx, maxy = poligono.bounds
        puntos = []
        while len(puntos) < cantidad:
            random_point = Point(rng.uniform(minx, maxx), rng.uniform(miny, maxy))
            if poligono.contains(random_point):
                puntos.append(random_point)
        return puntos

    # Función para calcular la distancia Haversine en metros
    def distancia_haversine(punto1, punto2):
        # Radio de la Tierra en metros
        R = 6371000
        lat1 = math.radians(punto1.y)
        lon1 = math.radians(punto1.x)
        lat2 = math.radians(punto2.y)
        lon2 = math.radians(punto2.x)

        dlat = lat2 - lat1
        dlon = lon2 - lon1

        a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

        distancia = R * c
        return distancia  # Retorna la distancia en metros

    # Generar ubicaciones aleatorias para repas y envíos
    puntos_repas = generar_puntos_en_poligono(poligono, cantidad_repas)
    puntos_envios = generar_puntos_en_poligono(poligono, cantidad_envios)

    # Asignar IDs
    ids_repas = list(range(len(puntos_repas)))
    ids_envios = list(range(len(puntos_envios)))

    # Combinar todos los puntos para clustering
    puntos_todos = puntos_repas + puntos_envios
    etiquetas = ['repa'] * len(puntos_repas) + ['envio'] * len(puntos_envios)
    ids_todos = ids_repas + ids_envios  # IDs correspondientes a puntos_todos

    # Obtener coordenadas para clustering
    coordenadas = np.array([[p.x, p.y] for p in puntos_todos])

    # Realizar clustering inicial
    print(f"\n--- Primera Agrupación con {n_clusters} clusters ---")
    n_clusters_current = min(n_clusters, len(coordenadas))
    kmeans = KMeans(n_clusters=n_clusters_current, random_state=random_state)
    clusters = kmeans.fit_predict(coordenadas)

    # Inicializar listas de asignaciones y sobrantes
    asignaciones = []
    sobrantes_repas = ids_repas.copy()
    sobrantes_envios = ids_envios.copy()

    distancia_total_metros = 0  # Para acumular la distancia total de las asignaciones en metros

    # Procesar cada cluster
    for cluster_id in range(n_clusters_current):
        # Obtener índices de puntos en el cluster actual
        indices_cluster = np.where(clusters == cluster_id)[0]

        # Separar repas y envíos en el cluster
        repas_cluster = []
        envios_cluster = []
        ids_repas_cluster = []
        ids_envios_cluster = []

        for idx in indices_cluster:
            etiqueta = etiquetas[idx]
            if etiqueta == 'repa':
                repas_cluster.append(puntos_todos[idx])
                ids_repas_cluster.append(ids_todos[idx])
            else:
                envios_cluster.append(puntos_todos[idx])
                ids_envios_cluster.append(ids_todos[idx])

        # Mostrar información del cluster
        print(f"\nCluster {cluster_id}:")
        print(f" - Repartidores en el cluster: {len(repas_cluster)}")
        print(f" - Envíos en el cluster: {len(envios_cluster)}")

        # Si no hay repas o envíos en el cluster, continuar
        if len(repas_cluster) == 0 or len(envios_cluster) == 0:
            print(" - Cluster sin asignaciones posibles.")
            continue

        # Crear matriz de distancias utilizando paralelización
        def calcular_distancias_par(i):
            repa = repas_cluster[i]
            return [distancia_haversine(repa, envio) for envio in envios_cluster]

        resultados = Parallel(n_jobs=n_jobs)(
            delayed(calcular_distancias_par)(i) for i in range(len(repas_cluster))
        )

        matriz_distancias = np.array(resultados)

        # Resolver el problema de asignación utilizando el algoritmo húngaro
        filas_ind, columnas_ind = linear_sum_assignment(matriz_distancias)

        num_asignaciones_cluster = min(len(filas_ind), len(columnas_ind))
        print(f" - Asignaciones en este cluster: {num_asignaciones_cluster}")

        # Acumular distancia total y realizar asignaciones
        for idx in range(num_asignaciones_cluster):
            i = filas_ind[idx]
            j = columnas_ind[idx]
            repa_id = ids_repas_cluster[i]
            envio_id = ids_envios_cluster[j]
            asignaciones.append((repa_id, envio_id))
            distancia_total_metros += matriz_distancias[i, j]
            # Remover asignados de los sobrantes
            if repa_id in sobrantes_repas:
                sobrantes_repas.remove(repa_id)
            if envio_id in sobrantes_envios:
                sobrantes_envios.remove(envio_id)

    # Mostrar estado después de la primera agrupación
    print(f"\n--- Estado después de la primera agrupación ---")
    print(f"Total de asignaciones: {len(asignaciones)}")
    print(f"Repartidores sobrantes: {len(sobrantes_repas)}")
    print(f"Envíos sobrantes: {len(sobrantes_envios)}")

    # Si hay sobrantes, volver a clusterizar
    iteracion = 1
    asignaciones_en_iteracion = len(asignaciones)
    while sobrantes_repas and sobrantes_envios and n_clusters > 1:
        iteracion += 1

        # Si no hubo asignaciones en la iteración anterior, reducir el número de clusters
        if asignaciones_en_iteracion == 0:
            n_clusters -= 1
            print(f"\nNo hubo asignaciones en la iteración anterior. Reduciendo clusters a {n_clusters}")

        asignaciones_en_iteracion = 0  # Reiniciar el contador de asignaciones en esta iteración

        print(f"\n--- Reagrupación de sobrantes: Iteración {iteracion} ---")

        # Actualizar puntos y etiquetas de sobrantes
        puntos_repas_sobrantes = [puntos_repas[ids_repas.index(i)] for i in sobrantes_repas]
        puntos_envios_sobrantes = [puntos_envios[ids_envios.index(i)] for i in sobrantes_envios]

        ids_repas_sobrantes = sobrantes_repas.copy()
        ids_envios_sobrantes = sobrantes_envios.copy()

        puntos_todos_sobrantes = puntos_repas_sobrantes + puntos_envios_sobrantes
        etiquetas_sobrantes = ['repa'] * len(puntos_repas_sobrantes) + ['envio'] * len(puntos_envios_sobrantes)
        ids_todos_sobrantes = ids_repas_sobrantes + ids_envios_sobrantes

        # Actualizar coordenadas
        coordenadas_sobrantes = np.array([[p.x, p.y] for p in puntos_todos_sobrantes])

        # Calcular el número actual de clusters
        n_clusters_current = min(n_clusters, len(coordenadas_sobrantes))
        print(f" - Número de clusters en esta iteración: {n_clusters_current}")

        # Evitar clusters vacíos
        if n_clusters_current <= 0:
            break

        # Re-clusterizar sobrantes
        kmeans = KMeans(n_clusters=n_clusters_current, random_state=random_state)
        clusters_sobrantes = kmeans.fit_predict(coordenadas_sobrantes)

        # Procesar cada cluster de sobrantes
        for cluster_id in range(n_clusters_current):
            indices_cluster = np.where(clusters_sobrantes == cluster_id)[0]

            repas_cluster = []
            envios_cluster = []
            ids_repas_cluster = []
            ids_envios_cluster = []

            for idx in indices_cluster:
                etiqueta = etiquetas_sobrantes[idx]
                if etiqueta == 'repa':
                    repas_cluster.append(puntos_todos_sobrantes[idx])
                    ids_repas_cluster.append(ids_todos_sobrantes[idx])
                else:
                    envios_cluster.append(puntos_todos_sobrantes[idx])
                    ids_envios_cluster.append(ids_todos_sobrantes[idx])

            # Mostrar información del cluster
            print(f"\nCluster {cluster_id}:")
            print(f" - Repartidores en el cluster: {len(repas_cluster)}")
            print(f" - Envíos en el cluster: {len(envios_cluster)}")

            if len(repas_cluster) == 0 or len(envios_cluster) == 0:
                print(" - Cluster sin asignaciones posibles.")
                continue

            # Crear matriz de distancias utilizando paralelización
            resultados = Parallel(n_jobs=n_jobs)(
                delayed(calcular_distancias_par)(i) for i in range(len(repas_cluster))
            )

            matriz_distancias = np.array(resultados)

            filas_ind, columnas_ind = linear_sum_assignment(matriz_distancias)

            num_asignaciones_cluster = min(len(filas_ind), len(columnas_ind))
            print(f" - Asignaciones en este cluster: {num_asignaciones_cluster}")

            # Acumular distancia total y realizar asignaciones
            for idx in range(num_asignaciones_cluster):
                i = filas_ind[idx]
                j = columnas_ind[idx]
                repa_id = ids_repas_cluster[i]
                envio_id = ids_envios_cluster[j]
                asignaciones.append((repa_id, envio_id))
                distancia_total_metros += matriz_distancias[i, j]
                asignaciones_en_iteracion += 1
                if repa_id in sobrantes_repas:
                    sobrantes_repas.remove(repa_id)
                if envio_id in sobrantes_envios:
                    sobrantes_envios.remove(envio_id)

        # Mostrar estado después de la iteración
        print(f"\nEstado después de la iteración {iteracion}:")
        print(f"Total de asignaciones: {len(asignaciones)}")
        print(f"Repartidores sobrantes: {len(sobrantes_repas)}")
        print(f"Envíos sobrantes: {len(sobrantes_envios)}")

    total_asignaciones = len(asignaciones)
    return asignaciones, sobrantes_repas, sobrantes_envios, total_asignaciones, distancia_total_metros

In [2]:
vertices= ((-34.6251522,-58.4132406),
(-34.6208589,-58.379557),
(-34.6149255,-58.3792137),
(-34.5968403,-58.3994698),
(-34.5745109,-58.3960365),
(-34.5452476,-58.4434151),
(-34.5682916,-58.4669327),
(-34.5686057,-58.49595),
(-34.5946108,-58.505563),
(-34.6233159,-58.4879133),
(-34.6398418,-58.4599325),
(-34.6320736,-58.4448263),
(-34.6251522,-58.4132406))

In [10]:
asignar_repas_envios(vertices, 500, 500, 9, random_state=123, n_jobs=-1)


--- Primera Agrupación con 9 clusters ---

Cluster 0:
 - Repartidores en el cluster: 50
 - Envíos en el cluster: 39
 - Asignaciones en este cluster: 39

Cluster 1:
 - Repartidores en el cluster: 71
 - Envíos en el cluster: 50
 - Asignaciones en este cluster: 50

Cluster 2:
 - Repartidores en el cluster: 58
 - Envíos en el cluster: 53
 - Asignaciones en este cluster: 53

Cluster 3:
 - Repartidores en el cluster: 55
 - Envíos en el cluster: 49
 - Asignaciones en este cluster: 49

Cluster 4:
 - Repartidores en el cluster: 61
 - Envíos en el cluster: 71
 - Asignaciones en este cluster: 61

Cluster 5:
 - Repartidores en el cluster: 51
 - Envíos en el cluster: 66
 - Asignaciones en este cluster: 51

Cluster 6:
 - Repartidores en el cluster: 48
 - Envíos en el cluster: 55
 - Asignaciones en este cluster: 48

Cluster 7:
 - Repartidores en el cluster: 44
 - Envíos en el cluster: 49
 - Asignaciones en este cluster: 44

Cluster 8:
 - Repartidores en el cluster: 62
 - Envíos en el cluster: 68
 - 

([(1, 420),
  (8, 138),
  (11, 318),
  (17, 206),
  (25, 471),
  (41, 121),
  (75, 447),
  (81, 398),
  (86, 273),
  (123, 247),
  (124, 324),
  (125, 222),
  (132, 146),
  (133, 452),
  (143, 411),
  (162, 294),
  (164, 58),
  (175, 264),
  (182, 395),
  (196, 142),
  (215, 162),
  (221, 171),
  (235, 75),
  (250, 1),
  (253, 275),
  (260, 214),
  (304, 235),
  (332, 212),
  (354, 44),
  (355, 36),
  (359, 342),
  (377, 117),
  (394, 33),
  (433, 196),
  (458, 236),
  (462, 341),
  (465, 415),
  (481, 165),
  (486, 150),
  (9, 175),
  (26, 498),
  (30, 229),
  (31, 385),
  (35, 128),
  (45, 485),
  (53, 479),
  (54, 329),
  (93, 254),
  (101, 198),
  (115, 124),
  (130, 451),
  (147, 199),
  (154, 289),
  (189, 68),
  (200, 223),
  (201, 224),
  (213, 213),
  (216, 276),
  (225, 95),
  (230, 148),
  (245, 131),
  (246, 176),
  (247, 149),
  (248, 57),
  (277, 151),
  (284, 217),
  (302, 304),
  (305, 383),
  (331, 290),
  (336, 34),
  (338, 403),
  (340, 26),
  (343, 99),
  (365, 243)